Here, we demonstrate how computationally intensive subroutines can escape the [Global Interpreter Lock](https://realpython.com/python-gil/). What ends up happening here is that the computational solve routines run efficiently in parallel, because Python can thread switch while the `gurobipy.Model.optimize` subroutine is running. This is possible because the Gurobi kernel is written in C, and thus the thread switching can happen without violating the GIL.

In general, parallelization of computational code can only be achieved with Python by using `multiprocessing`, as a result of the GIL. The exception to that rule is when the performance bottleneck is in non-Python libraries.

See `no_concurrency_at_all.ipynb` for a benchmark of what the performance is like without parallelization for these same MIP models. See `concurrent_multiprocessing.ipynb` for the same performance improvement seen here, except using `multiprocessing`.

In [1]:
import time
import cogmodel
from ticdat import Progress, LogFile
from threading import Thread
import os

Because this machine has so few cores, the most efficient use of the cores we have is to restrict Gurobi to only use one thread for MIP solve, and to use `threading` to run the MIP solves in parallel. 

In [2]:
print(os.cpu_count()) 

4


In [3]:
dat = cogmodel.input_schema.json.create_tic_dat("cog_sample_data.json")
dat.parameters["Gurobi Threads"] = 1
dat.parameters["Gurobi LogToConsole"] = 0

In [4]:
dat_2 = cogmodel.input_schema.copy_tic_dat(dat)
dat_2.parameters["Number of Centroids"] = 4

In [5]:
dat_3 = cogmodel.input_schema.copy_tic_dat(dat)
dat_3.parameters["Number of Centroids"] = 5

In [6]:
 # feel free to suggest a cleaner way to buffer the inputs/outputs from the threads
d_s_bfr = ([dat, None], [dat_2, None], [dat_3, None])

In [7]:
def task(id):
    sln = cogmodel.solve(d_s_bfr[id][0], LogFile(None), LogFile(None), Progress(quiet=True))
    d_s_bfr[id][1] = sln

In [8]:
start = time.time()
threads = []
for i in range(len(d_s_bfr)):
    t = Thread(target=task, args=(i,))
    threads.append(t)
    t.start()

In [9]:
for t in threads:
    t.join()

Using license file /Users/petercacioppi/gurobi.lic
Using license file /Users/petercacioppi/gurobi.lic
Using license file /Users/petercacioppi/gurobi.lic
Changed value of parameter Threads to 1
Changed value of parameter Threads to 1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
   Prev: 0  Min: 0  Max: 1024  Default: 0
   Prev: 0  Min: 0  Max: 1024  Default: 0


In [10]:
print(f"\n\n****\nRequired {time.time() - start} seconds in total")



****
Required 37.50310301780701 seconds in total


In [11]:
for d, s in d_s_bfr:
    assert d.parameters["Number of Centroids"]["Value"] == len(s.openings)